# Take a Pre-Trained Network (ResNet50) and look at some filters

In [1]:
# Import Required modules

from keras.preprocessing import image
from keras.applications import resnet50
from keras import models
import keras.backend as K

import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, clear_output



Using TensorFlow backend.


In [2]:
# Load Keras' ResNet50 model that was pre-trained against the ImageNet database
model = resnet50.ResNet50()
layer_outputs = [layer.output for layer in model.layers] 
activation_model = models.Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Print the summary of the model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
# Check that everything is working as expected

# Load the image file, resizing it to 224x224 pixels (required by this model)
img = image.load_img("DSC_2631.JPG", target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add a forth dimension since Keras expects a list of images
x = np.expand_dims(x, axis=0)

# Scale the input image to the range used in the trained network
x = resnet50.preprocess_input(x)

# Run the image through the deep neural network to make a prediction
predictions = model.predict(x)

# Look up the names of the predicted classes. Index zero is the results for the first image.
predicted_classes = resnet50.decode_predictions(predictions, top=9)

print("This is an image of:")

for imagenet_id, name, likelihood in predicted_classes[0]:
    print(" - {}: {:2f} likelihood".format(name, likelihood))


This is an image of:
 - Labrador_retriever: 0.237099 likelihood
 - bloodhound: 0.126507 likelihood
 - Weimaraner: 0.120869 likelihood
 - Great_Dane: 0.118317 likelihood
 - basset: 0.075523 likelihood
 - Chesapeake_Bay_retriever: 0.043061 likelihood
 - ice_bear: 0.040971 likelihood
 - redbone: 0.035126 likelihood
 - golden_retriever: 0.034954 likelihood


In [4]:
# Map the layer outputs and create a function that returns the result (activations) of those layers for a given input
layer_outputs = [K.identity(layer.output) for layer in model.layers]
activation_f = K.function([model.input], layer_outputs)


In [5]:
# Define some helpers

def clean_filter(channel_image):
    channel_image -= np.mean(channel_image).astype('uint8')#.mean() # Post-processes the feature to make it visually palatable
    channel_image = channel_image / np.std(channel_image).astype('uint8')#.std()
    channel_image *= 64
    channel_image += 128
    channel_image = np.clip(channel_image, 0, 255).astype('uint8')
    channel_image = cv2.resize(channel_image, (224, 224))
    return channel_image

# Takes a bunch of activations and tiles them nicely for plotting
def grid_activations(layer_activation,images_per_row):
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    return cv2.convertScaleAbs(display_grid)


In [9]:
# Take webcam captures and plot their activations

cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 
ret,frame = cap.read() # return a single frame in variable `frame`

layer_num = 40 # Change this to play around with various layers

while(True):
    ret,frame = cap.read() # return a single frame in variable `frame
    # Add a forth dimension since Keras expects a list of images
    x = cv2.resize(frame, (224, 224)) 
    x = np.expand_dims(x, axis=0)

    # Scale the input image to the range used in the trained network
    x = resnet50.preprocess_input(x)

    # Run the image through our activation function
    activations = activation_f([x])
    filter_img=clean_filter(activations[layer_num][0,:, :,20*2+2])
    x = cv2.resize(frame, (x.shape[1],x.shape[2]))
    # Give the filter 3 dimensions and replicate it across those three dimensions
    filter_img=np.expand_dims(filter_img,axis=2)
    filter_img=np.tile(filter_img,[1,1,3])
    
    cv2.imshow(model.layers[layer_num].name,np.concatenate([x,filter_img])) #display the captured image overtop the filters
    
    # If you want to plot all the filters tiled:
    # cv2.imshow(model.layers[layer_num].name,grid_activations(activations[layer_num],20)) #display tiled filters from layer
        
    if cv2.waitKey(1) & 0xFF == ord('y'): #Stop on pressing 'y' 
        cv2.destroyAllWindows()
        break
cap.release()


In [7]:
# In case something errors out, this will release the webcam
cap.release()

